In [11]:
import numpy as np
import pandas as pd
import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

In [47]:
# import dataset

df = pd.read_csv('data/1976-2020-president.csv')
df.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified
0,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,20210113,NaN,DEMOCRAT
1,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"FORD, GERALD",REPUBLICAN,False,504070,1182850,20210113,NaN,REPUBLICAN
2,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"MADDOX, LESTER",AMERICAN INDEPENDENT PARTY,False,9198,1182850,20210113,NaN,OTHER
3,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"BUBAR, BENJAMIN """"BEN""""",PROHIBITION,False,6669,1182850,20210113,NaN,OTHER
4,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"HALL, GUS",COMMUNIST PARTY USE,False,1954,1182850,20210113,NaN,OTHER


##### Exploratory analysis

[Project Master Document](https://docs.google.com/document/d/1gahC29lp4TdxrA2PnUVGcLdlwcdKvBaaSxkmxr35FZE/edit?usp=sharing})

Focus:
Incorporate economic indicators (make new variables or merge with external datasets)
- Analyze political preference over time against economic cycles
    - (e.g. inflation, peaks/troughs, mark the 2007-8 economic crisis, 9/11) for noteworthy trends/graphs/stats


In [174]:
# useful data structures

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct',\
    'Nov', 'Dec']

elecYears = [str(_) for _ in df['year'].unique()]

In [48]:
df.dtypes

year                  int64
state                object
state_po             object
state_fips            int64
state_cen             int64
state_ic              int64
office               object
candidate            object
party_detailed       object
writein              object
candidatevotes        int64
totalvotes            int64
version               int64
notes               float64
party_simplified     object
dtype: object

In [49]:
# import economic datasets

cpia = pd.read_excel('data/cpi/cpi-all-items.xlsx', header=11) 
cpig = pd.read_excel('data/cpi/cpi-gasoline.xlsx', header=11)
cpih = pd.read_excel('data/cpi/cpi-housing.xlsx', header=11)
cpim = pd.read_excel('data/cpi/cpi-medical-care.xlsx', header=11)

In [50]:
# sanity check
print(df['office'].unique())
print()
print(df['writein'].value_counts())
print()
print(df['party_simplified'].value_counts())
print()
print(df['notes'].value_counts())

['US PRESIDENT']

False    3807
True      477
Name: writein, dtype: int64

OTHER          2524
DEMOCRAT        615
REPUBLICAN      613
LIBERTARIAN     535
Name: party_simplified, dtype: int64

Series([], Name: notes, dtype: int64)


In [51]:
df.drop(columns={'state_po', 'state_fips', 'state_cen', 'state_ic', 'office', \
    'party_detailed', 'version', 'notes'}, inplace=True)

df.rename(columns={'party_simplified': 'party'}, inplace=True)

for col in df.columns:
    if isinstance(df[col][0], str):
        df[col] = df[col].str.lower()

In [52]:
df.head(3)

,year,state,candidate,writein,candidatevotes,totalvotes,party
0,1976,alabama,"carter, jimmy",False,659170,1182850,democrat
1,1976,alabama,"ford, gerald",False,504070,1182850,republican
2,1976,alabama,"maddox, lester",False,9198,1182850,other


In [53]:
cpia.head(3)

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,HALF1,HALF2
0,1976.0,55.6,55.8,55.9,56.1,56.5,56.8,57.1,57.4,57.6,57.9,58.0,58.2,NaN,NaN
1,1977.0,58.5,59.1,59.5,60.0,60.3,60.7,61.0,61.2,61.4,61.6,61.9,62.1,NaN,NaN
2,1978.0,62.5,62.9,63.4,63.9,64.5,65.2,65.7,66.0,66.5,67.1,67.4,67.7,NaN,NaN


In [60]:
# get majority party preference by year

# add a column showing supporting percentage for each party each election year
# pp ~ party preference
pp = df.groupby(['year', 'party'])[['candidatevotes', 'totalvotes']].sum()
pp['support'] = np.round(pp['candidatevotes'] / pp['totalvotes'], 3)
pp.head()

candidatevotes  totalvotes  support
year party                                           
1976 democrat           40680446    81601344    0.499
     libertarian           95626    37855605    0.003
     other               1954379   404885771    0.005
     republican         38870893    81601344    0.476
1980 democrat           35480948    86496851    0.410

In [63]:
cpia.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,HALF1,HALF2
0,1976.0,55.6,55.8,55.9,56.1,56.5,56.8,57.1,57.4,57.6,57.9,58.0,58.2,NaN,NaN
1,1977.0,58.5,59.1,59.5,60.0,60.3,60.7,61.0,61.2,61.4,61.6,61.9,62.1,NaN,NaN
2,1978.0,62.5,62.9,63.4,63.9,64.5,65.2,65.7,66.0,66.5,67.1,67.4,67.7,NaN,NaN
3,1979.0,68.3,69.1,69.8,70.6,71.5,72.3,73.1,73.8,74.6,75.2,75.9,76.7,NaN,NaN
4,1980.0,77.8,78.9,80.1,81.0,81.8,82.7,82.7,83.3,84.0,84.8,85.5,86.3,NaN,NaN


In [81]:
str(cpia['Year'][0])[:-2]

'1976'

In [175]:
def getAvgCPI(df):
    
    """ 
        Get average CPI per year

    """ 

    output = df.drop(columns={'HALF1', 'HALF2'})
    output.set_index('Year', drop=True, inplace=True)
    output = output.T.apply(np.mean, axis=0)
    output = pd.DataFrame(data=output)
    output.reset_index(inplace=True)
    output['Year'] = [str(_)[:-2] for _ in output['Year']]
    output.rename(columns={0:'cpi'}, inplace=True)
    
    return output[output['Year'].isin(elecYears)]

In [176]:
cpiaNew = getAvgCPI(cpia)
cpiaNew.head()

,Year,cpi
0,1976,56.908333
4,1980,82.408333
8,1984,103.883333
12,1988,118.258333
16,1992,140.316667


In [177]:
pp.head(5)

candidatevotes  totalvotes  support
year party                                           
1976 democrat           40680446    81601344    0.499
     libertarian           95626    37855605    0.003
     other               1954379   404885771    0.005
     republican         38870893    81601344    0.476
1980 democrat           35480948    86496851    0.410

In [178]:
# sanity check
pp.loc[1976]

,candidatevotes,totalvotes,support
party,,,
democrat,40680446,81601344,0.499
libertarian,95626,37855605,0.003
other,1954379,404885771,0.005
republican,38870893,81601344,0.476


In [180]:
# Merge cpi data with elections data (pp)

def mergeCPI(cpiData, elecData):
    
    # merge dem data
    mergeDem = lambda row: elecData.loc[int(row['Year'])].loc['democrat']['support']
    cpiData['democrat support'] = cpiData.apply(mergeDem, axis=1)

    print(cpiData)  

In [181]:
mergeCPI(cpiaNew, pp)

    Year         cpi  democrat support
0   1976   56.908333             0.499
4   1980   82.408333             0.410
8   1984  103.883333             0.404
12  1988  118.258333             0.455
16  1992  140.316667             0.429
20  1996  156.850000             0.491
24  2000  172.200000             0.481
28  2004  188.883333             0.472
32  2008  215.302500             0.528
36  2012  229.593917             0.509
40  2016  240.007167             0.462
44  2020  258.811167             0.513


In [ ]:
# Unused code archive

# def restructureCPIData(df):
    
#     """ 
#         Reorganize the cpi datasets to combine the months and years into one
#         variable in datetime type. Each row is now associated with only one 
#         cpi number.

#     """ 

#     rowsModified = []

#     def processOneRow(row):
#         timeline, data = [], []
#         year = row['Year']

#         for m in months:
#             time = pd.to_datetime(str(year)[:-2]+' '+m)
#             datum = row[m]
#             timeline.append(time)
#             data.append(datum)
        
#         rowsModified.append(pd.DataFrame(data={'timeline':timeline,'cpi': data}))
    
#     df.apply(processOneRow, axis=1)

#     return pd.concat(rowsModified)